### --- Day 12: Garden Groups ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/12

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 12);
WriteLines(inputLines);

Loading puzzle file: Day12.txt
Total lines: 140
Max line length: 140

RRRRRRRRRRRRRRRRKKKKKKKKRRRRRRRRRRRRRRRRRRRROOPOOOOOOOOGOOONYYYYYYEDDDDDDDDDDDLLLLLLLLOOOOOOOOOOOOOOOXVVXHHHHHHHHHHLLLLLLLLLLLLLLJJJJJJJJJJC
RRRRRRRRRRRRRRRRKKKKKKKKKRRRRRRRRRRRRRRRRRRRROOOOOOOOOOOOOOYXYYYYKEDDDDDDDIIIDLLLLLLLLLLOOOOOOOOOOOOOXXXXHHHHHHHLLLLLLLLLLLLLLLLJJJJJJJJJJJC
RRRRRRRRRRRRRRKKKKKKKKKKRRRRRRRRRRRRRRRRRRRRROOOOOOOOOOOOOOYYYYYYKEDDDDDDDDDILLLLLLLLLLLOOOOOOOOOOOOOOXXXXXXXHHHHHLLLLLLLLLLLLLJJJJJJJJJJJJC
RRRRRRRRRRRRKKKKKKKKKKKKKRRRRRRRRRRRRRRRRRRRROOOOOOOOOOOOOOOOYYYYKDDDDDDDDDDIILLLLLLLLLLOOOOOOOOOOOOOOXXXXXXXHHHHHLLLLLLLLLLLLLLJJJJJJJJJJJC
RRRRRRRRRRRRKKKKKKKKKKKKKERREREERRRRRMMMRRRROOOOOOOOOOOOOOOOYYYYYKKDDDDDDDDDDDDLLLLLLLLOOOOOOOOOOOOOOOXXXXXXXHHHHLLLLLLLLLLLLLLJJJJJJJJJJAAA


In [4]:
string[] testInputLinesSmall = [
    "AAAA",
    "BBCD",
    "BBCC",
    "EEEC",
];

Let's model the group: a collection of points with the same plant type.

In [5]:
class Group
{
    public Group(char plantType, Point initialMember)
    {
        PlantType = plantType;
        Members.Add(initialMember);
    }

    public char PlantType { get; }
    public HashSet<Point> Members { get; } = new();
}

We need to find the groups. We can do this by picking a point and walking the graph for neighbours of the same type:

In [6]:
IEnumerable<Group> FindGroups(CharGrid grid)
{
    HashSet<Point> grouped = new();
    var getNeighbours = (Point origin) => UDLR.Select(dir => origin + dir).Where(p => grid.IsValid(p));

    foreach (var (point, ch) in grid.Enumerate())
    {
        if (grouped.Contains(point)) { continue; }

        Group newGroup = new(ch, point);

        var getMatchingNeighbours = (Point origin) => getNeighbours(origin).Where(n => grid[n] == newGroup.PlantType);
        foreach (var neighbour in BFS(point, getMatchingNeighbours))
        {
            newGroup.Members.Add(neighbour);
            grouped.Add(neighbour);
        }

        yield return newGroup;
    }
}

Point[] UDLR = [Up, Down, Left, Right];


Now that we have the groups, the rest is fairly straightforward:

In [7]:
int GetTotalPrice(string[] inputLines)
{
    CharGrid grid = new(inputLines);
    var groups = FindGroups(grid);

    return groups.Select(GetPrice).Sum();
}

int GetPrice(Group group)
{
    var area = group.Members.Count;
    var perimeter = area * 4;

    var internalEdges = from point in @group.Members
                        from neighbour in UDLR.Select(dir => point + dir)
                        where @group.Members.Contains(neighbour)
                        select 1;

    perimeter -= internalEdges.Sum();

    return area * perimeter;
}

In [8]:
// In the first example, region A has price 4 * 10 = 40, region B has price 4 *
// 8 = 32, region C has price 4 * 10 = 40, region D has price 1 * 4 = 4, and region
// E has price 3 * 8 = 24. So, the total price for the first example is 140.

var smallTestAnswer = GetTotalPrice(testInputLinesSmall);
Console.WriteLine(smallTestAnswer);

140


In [9]:
// In the second example, the region with all of the O plants has price 21 * 36
// = 756, and each of the four smaller X regions has price 1 * 4 = 4, for a total
// price of 772 (756 + 4 + 4 + 4 + 4).

string[] testInputLinesX = [
    "OOOOO",
    "OXOXO",
    "OOOOO",
    "OXOXO",
    "OOOOO",
];
var xTestAnswer = GetTotalPrice(testInputLinesX);
Console.WriteLine(xTestAnswer);

772


In [10]:
// Here's a larger example... So, it has a total price of 1930.

string[] testInputLinesLarge = [
    "RRRRIICCFF",
    "RRRRIICCCF",
    "VVRRRCCFFF",
    "VVRCCCJFFF",
    "VVVVCJJCFE",
    "VVIVCCJJEE",
    "VVIIICJJEE",
    "MIIIIIJJEE",
    "MIIISIJEEE",
    "MMMISSJEEE",
];

var largeTestAnswer = GetTotalPrice(testInputLinesLarge);
Console.WriteLine(largeTestAnswer);

1930


In [11]:
// What is the total price of fencing all regions on your map?

var part1Answer = GetTotalPrice(inputLines);
Console.WriteLine(part1Answer);

1375476


In [12]:
// 1375476 is correct!
Ensure(1375476, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/12

We know a member of a group has a top fence if the neighbour above it is not part of our group. Similarly for bottom, left, right.

Sticking with top, we know fence is part of a side if it's adjacent to another fence on the same row. If there's a gap, or if it's a different row, they can't possibly be part of the same side.

Again we can apply similar logic for bottom, left, right.

In [14]:
int GetTotalPrice2(string[] inputLines)
{
    CharGrid grid = new(inputLines);
    var groups = FindGroups(grid);

    return groups.Select(GetPrice2).Sum();
}

int GetPrice2(Group group)
{
    var area = group.Members.Count;

    int totalSideCount = 0;
    totalSideCount += CountSides(group, Up);
    totalSideCount += CountSides(group, Down);
    totalSideCount += CountSides(group, Left, transpose: true);
    totalSideCount += CountSides(group, Right, transpose: true);

    return totalSideCount * area;
}

int CountSides(Group group, Point direction, bool transpose = false)
{
    var points = group.Members.Where(point => !group.Members.Contains(point + direction));
    if (transpose) {
        points = points.Select(p => new Point(p.Y, p.X));
    }

    var orderedPoints = points.OrderBy(p => (p.Y, p.X)).ToList();
    return 1 + orderedPoints.Zip(orderedPoints[1..]).Where(op => (op.Second - op.First != Right)).Count();
}

In [15]:
// Here's a map that includes an E-shaped region full of type E plants:
// ...
// Including the two regions full of type X plants, this map has a total price of 236.

string[] testInputE = [
    "EEEEE",
    "EXXXX",
    "EEEEE",
    "EXXXX",
    "EEEEE",
];
var part2TestAnswerE = GetTotalPrice2(testInputE);
Console.WriteLine(part2TestAnswerE);

236


In [16]:
// This map has a total price of 368:

string[] testInputF = [
    "AAAAAA",
    "AAABBA",
    "AAABBA",
    "ABBAAA",
    "ABBAAA",
    "AAAAAA",
];
var part2TestAnswerF = GetTotalPrice2(testInputF);
Console.WriteLine(part2TestAnswerF);

368


In [17]:
// The larger example from before now has the following updated prices:
// ...
// Adding these together produces its new total price of 1206.

var part2LargeTestAnswer = GetTotalPrice2(testInputLinesLarge);
Console.WriteLine(part2LargeTestAnswer);

1206


In [18]:
// What is the new total price of fencing all regions on your map?

var part2Answer = GetTotalPrice2(inputLines);
Console.WriteLine(part2Answer);

821372


In [19]:
// 821372 is correct!
Ensure(821372, part2Answer);